# Municipality Analysis

In [0]:
#Cargando librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/files/nutritive.csv')
df = df.drop(columns=['Unnamed: 0'])

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
from unicodedata import normalize
def cleanText(inputString):
    return (re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", inputString), 0, re.I
    )).lower()

def clean_l3(row):
    row.l3 = cleanText(row.l3)
    return row

def clean_alpha(inputString):
    return re.sub('[^A-Za-z0-9]+', '', inputString)

## Sell and Rent Counting

In [5]:
df = df.where(df['operation_type'] == 'Venta').dropna(subset=['operation_type'])
df = df.apply(lambda x: clean_l3(x), axis=1)
df.shape

(613054, 25)

In [0]:
df_train = df.copy()
    
df_train = df_train[df_train['price']<df_train['price'].quantile(0.99)]
df_train = df_train[df_train['price'] > df_train['price'].quantile(0.1)]
df_train = df_train[df_train['surface_total'] > 0]
df_train = df_train[~np.isnan(df_train['price']) & df_train['price'] > 0]
df_train = df_train[df_train['surface_total'] > 0]

df_train['price'] = np.log(df_train['price'])

In [7]:
df_train.head(1)

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,HSOmvzoNx39wqCR/yL57kw==,Propiedad,2018-09-14,2019-09-23,2018-09-14,3.439,-76.541,Colombia,Valle del Cauca,cali,NaN,NaN,NaN,12.0,12.0,6.0,202.0,360.0,20.030119,COP,NaN,Casa En Venta En Cali Miraflores,"Para inversin, actualmente rentando, casa de 4...",Casa,Venta


In [0]:
df_final = df_train.copy()

In [0]:
n = 10 #First n most frequent municipalities
series = df_train['l3'].value_counts()[:n]
municipalities = list(series.index)
freq = list(series)

In [0]:
df_balanced = pd.DataFrame(columns=list(df_train.columns))

In [0]:
#To make undersampling
n_under = min(freq) #Undersampling min number
df_final = df_train[df_train['l3'].isin(municipalities)]

for x in municipalities:
    temp_df = df_final[df_final['l3'] == x].sample(n = n_under, random_state=4)
    df_balanced = pd.concat([df_balanced, temp_df], ignore_index=True)
    

In [0]:
df_balanced.drop(columns=['id',"ad_type",'start_date', 'start_date','created_on','end_date',
                 'l1','l2', 'l4','l6', 'l5', 
                 'rooms', 'price_period', 'surface_covered',
                 'operation_type','currency', 'title', 'description'], inplace=True)

In [0]:
df_train = df_balanced.copy()

## Machine Learning - Initial Model

In [0]:
X_df = df_train.drop('price', axis=1).copy()
Y_df = df_train['price'].copy()

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train_or, X_test_or, Y_train_or, Y_test_or = train_test_split(
    X_df, Y_df, test_size=0.2, random_state=5000)

In [0]:
# Para eliminar SettingWithCopyWarning

X_train = X_train_or.copy()
X_test = X_test_or.copy()
Y_train = Y_train_or.copy()
Y_test = Y_test_or.copy()

In [0]:
from sklearn.preprocessing import (PowerTransformer, StandardScaler, 
                                   MinMaxScaler, LabelEncoder, OneHotEncoder)

### Power Transformer Labels (Box-Cox)

Normalize and reduce scale

In [21]:
box_cox = PowerTransformer(method='box-cox') # Box-Cox object instance
box_cox.fit(X_train["surface_total"].values.reshape(-1,1)) # Box-Cox, array 2 dimensions
X_train.loc[:,"surface_total"] = box_cox.transform(X_train["surface_total"].values.reshape(-1,1)) # Transformamos el array
X_train.head()

,lat,lon,l3,bedrooms,bathrooms,surface_total,property_type
64916,7.100000,-73.116997,bucaramanga,5.0,2.0,0.190543,Apartamento
20641,11.023675,-74.860947,barranquilla,NaN,3.0,-0.092228,Casa
29793,5.072902,-75.523653,manizales,4.0,2.0,0.861274,Casa
45451,7.893000,-72.488000,cucuta,2.0,2.0,-0.714284,Apartamento
62604,NaN,NaN,bucaramanga,4.0,4.0,0.768529,Apartamento


### Standard Scaler
El Standard Scaler sirve para centrar una distribución en 0 y que tenga desviación estándar unitaria.

 $$\mathbf{Z} = \frac{\mathbf{X} - \mathbf{\mu}}{\mathbf{\sigma}}$$


In [22]:
# Categorical Class
print("Classes: ", X_train['property_type'].unique())
X_train.head()

Classes:  ['Apartamento' 'Casa' 'Lote' 'Finca' 'Otro' 'Parqueadero' 'Oficina'
 'Local comercial' 'Depósito']


,lat,lon,l3,bedrooms,bathrooms,surface_total,property_type
64916,7.100000,-73.116997,bucaramanga,5.0,2.0,0.190543,Apartamento
20641,11.023675,-74.860947,barranquilla,NaN,3.0,-0.092228,Casa
29793,5.072902,-75.523653,manizales,4.0,2.0,0.861274,Casa
45451,7.893000,-72.488000,cucuta,2.0,2.0,-0.714284,Apartamento
62604,NaN,NaN,bucaramanga,4.0,4.0,0.768529,Apartamento


In [0]:
le_l3 = LabelEncoder()
le_l3.fit(X_train['l3'])
X_train['l3'] = le_l3.transform(X_train['l3'])

In [0]:
ohe_l3 = OneHotEncoder()
ohe_l3.fit(X_train["l3"].values[:,None])
out = ohe_l3.transform(X_train["l3"].values[:,None]) 
out_array = out.toarray()
ohe_cols_op = []
for category in ohe_l3.categories_[0]:
    current_class = le_l3.classes_[int(category)]
    col_add = "op_" + current_class
    ohe_cols_op.append(col_add)
    X_train[col_add] = out_array[:,int(category)]

X_train = X_train.drop("l3", axis =1)

In [0]:
le_proper_type = LabelEncoder()
le_proper_type.fit(X_train['property_type'])
X_train['property_type'] = le_proper_type.transform(X_train['property_type'])

In [0]:
ohe_proper_type = OneHotEncoder()
ohe_proper_type.fit(X_train["property_type"].values[:,None])
out = ohe_proper_type.transform(X_train["property_type"].values[:,None]) 
out_array = out.toarray()
ohe_cols_op2 = []
for category in ohe_proper_type.categories_[0]:
    current_class = le_proper_type.classes_[int(category)]
    col_add = "op_" + current_class
    ohe_cols_op2.append(col_add)
    X_train[col_add] = out_array[:,int(category)]

X_train = X_train.drop("property_type", axis =1)

### Tratamiento de datos faltantes o NaN

In [0]:
from sklearn.impute import SimpleImputer

In [0]:
# Using Simple Imputer
si_bedrooms = SimpleImputer(strategy='mean')
si_bedrooms.fit(X_train["bedrooms"].values[:,None])
X_train["bedrooms"] = si_bedrooms.transform(X_train["bedrooms"].values[:,None])

si_lat = SimpleImputer(strategy='mean')
si_lat.fit(X_train["lat"].values[:,None])
X_train["lat"] = si_lat.transform(X_train["lat"].values[:,None])

si_lon = SimpleImputer(strategy='mean')
si_lon.fit(X_train["lon"].values[:,None])
X_train["lon"] = si_lon.transform(X_train["lon"].values[:,None])

si_bath = SimpleImputer(strategy='mean')
si_bath.fit(X_train["bathrooms"].values[:,None])
X_train["bathrooms"] = si_bath.transform(X_train["bathrooms"].values[:,None])

In [29]:
#Otras transformaciones
cols2scale = ['bedrooms', 'bathrooms']
se_cols2scale = StandardScaler()
se_cols2scale.fit(X_train[cols2scale])
X_train[cols2scale] = se_cols2scale.transform(X_train[cols2scale])
X_train.head()

,lat,lon,bedrooms,bathrooms,surface_total,op_barranquilla,op_bogota d.c,op_bucaramanga,op_cali,op_cartagena,op_cucuta,op_envigado,op_manizales,op_medellin,op_pereira,op_Apartamento,op_Casa,op_Depósito,op_Finca,op_Local comercial,op_Lote,op_Oficina,op_Otro,op_Parqueadero
64916,7.100000,-73.116997,1.013676e+00,-0.605380,0.190543,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20641,11.023675,-74.860947,5.776931e-16,0.134606,-0.092228,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29793,5.072902,-75.523653,3.632511e-01,-0.605380,0.861274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45451,7.893000,-72.488000,-9.375981e-01,-0.605380,-0.714284,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62604,6.747057,-74.861314,3.632511e-01,0.874593,0.768529,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
X_test['surface_total'] = box_cox.transform(X_test['surface_total'].values[:,None])


In [0]:
X_test['property_type'] = le_proper_type.transform(X_test['property_type'])

In [0]:
X_test['l3'] = le_l3.transform(X_test['l3'])

In [0]:
temp_test = ohe_l3.transform(X_test["l3"].values[:,None]).toarray()
for idx, col in enumerate(ohe_cols_op):
    X_test[col] = temp_test[:,idx]

temp_test = ohe_proper_type.transform(X_test["property_type"].values[:,None]).toarray()

for idx, col in enumerate(ohe_cols_op2):
    X_test[col] = temp_test[:,idx]
    
    


X_test = X_test.drop("property_type", axis=1)
X_test = X_test.drop("l3", axis=1)

In [0]:
X_test['bedrooms'] = si_bedrooms.transform(X_test["bedrooms"].values[:,None])
X_test['lat'] = si_lat.transform(X_test['lat'].values[:, None])
X_test['lon'] = si_lon.transform(X_test['lon'].values[:, None])
X_test['bathrooms'] = si_bath.transform(X_test['bathrooms'].values[:,None])

In [0]:
X_test[cols2scale] = se_cols2scale.transform(X_test[cols2scale])

In [0]:
assert (X_test.columns.values == X_train.columns.values).all()

In [37]:
X_train

,lat,lon,bedrooms,bathrooms,surface_total,op_barranquilla,op_bogota d.c,op_bucaramanga,op_cali,op_cartagena,op_cucuta,op_envigado,op_manizales,op_medellin,op_pereira,op_Apartamento,op_Casa,op_Depósito,op_Finca,op_Local comercial,op_Lote,op_Oficina,op_Otro,op_Parqueadero
64916,7.100000,-73.116997,1.013676e+00,-0.605380,0.190543,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20641,11.023675,-74.860947,5.776931e-16,0.134606,-0.092228,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29793,5.072902,-75.523653,3.632511e-01,-0.605380,0.861274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45451,7.893000,-72.488000,-9.375981e-01,-0.605380,-0.714284,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62604,6.747057,-74.861314,3.632511e-01,0.874593,0.768529,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64898,7.092000,-73.111000,-2.871735e-01,-0.605380,-0.284716,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64641,7.120000,-73.112000,-2.871735e-01,0.134606,-0.339826,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55905,4.812486,-75.669839,5.776931e-16,-0.605380,-0.992690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34242,5.065670,-75.519837,5.776931e-16,-1.345367,-0.476691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [0]:
#Rename columns
new_col = []

for col in list(X_train.columns):
    new_col.append(clean_alpha(col))
    
    
X_train.columns = new_col
X_test.columns = new_col

## Measure of Error of the Models


In [0]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# measure error 
def error(Y_test, Y_estimate):
    rmse = mean_squared_error(Y_test, Y_estimate)
    mae = mean_absolute_error(Y_test, Y_estimate)
    r2 = r2_score(Y_test, Y_estimate)
    print("Root Mean Square Error (RMSE)= {:.3f}".format(rmse),
    "\nMean Absolute Error (MAE)= {:.3f}".format(mae),
    "\nR^2 = {:.3f}".format(r2))


In [0]:
from sklearn.model_selection import cross_val_score
# function to get cross validation scores
def get_cv_scores(model):
    scores = cross_val_score(model,
                             X_train,
                             Y_train,
                             cv=5,
                             scoring='r2')
    
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')

# Bagging

In [40]:
import sklearn
from sklearn.ensemble import BaggingRegressor
model1 = sklearn.ensemble.BaggingRegressor(n_estimators = 20,
                                           max_samples = 20, max_features = 24).fit(X_train, Y_train)

prediction1 = model1.predict(X_test)
error(Y_test, prediction1)

Root Mean Square Error (RMSE)= 0.275 
Mean Absolute Error (MAE)= 0.389 
R^2 = 0.557


## Bagging with ExtraTree

In [41]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
extra_tree = ExtraTreeRegressor(splitter = "best" ,random_state=1000)
model2 = BaggingRegressor(extra_tree, random_state=1000, n_estimators=60).fit(X_train, Y_train)
prediction2 = model2.predict(X_test)
error(Y_test, prediction2)

Root Mean Square Error (RMSE)= 0.126 
Mean Absolute Error (MAE)= 0.232 
R^2 = 0.798


In [0]:
# p = 2**Y_test
# p2 = 2**Y_test_estimate
# mae = mean_absolute_error(p, p2)
# print(mae)

## Lasso Regression

In [43]:
import sklearn
from sklearn.linear_model import  Lasso

model3 = sklearn.linear_model.Lasso(alpha = 0.1).fit(X_train, Y_train)
prediction3 = model3.predict(X_test)
error(Y_test, prediction3)

Root Mean Square Error (RMSE)= 0.338 
Mean Absolute Error (MAE)= 0.438 
R^2 = 0.456


## LightGBM

In [44]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt','objective': 'regression', 'metric': {'l2', 'l1'},'num_leaves': 31,
    'learning_rate': 0.05, 'feature_fraction': 0.7,'bagging_fraction': 0.8, 'bagging_freq': 5,  'verbose': 0
}

model4 = lgb.train(params,
                lgb_train,
                num_boost_round=6000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10000)

prediction4 = model4.predict(X_test, num_iteration=model4.best_iteration)
error(Y_test, prediction4)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[1007]	valid_0's l1: 0.256307	valid_0's l2: 0.132151
[1008]	valid_0's l1: 0.256304	valid_0's l2: 0.132148
[1009]	valid_0's l1: 0.256302	valid_0's l2: 0.132151
[1010]	valid_0's l1: 0.256287	valid_0's l2: 0.132154
[1011]	valid_0's l1: 0.256271	valid_0's l2: 0.132136
[1012]	valid_0's l1: 0.25628	valid_0's l2: 0.132159
[1013]	valid_0's l1: 0.256279	valid_0's l2: 0.132167
[1014]	valid_0's l1: 0.256268	valid_0's l2: 0.132169
[1015]	valid_0's l1: 0.256255	valid_0's l2: 0.132155
[1016]	valid_0's l1: 0.256251	valid_0's l2: 0.132154
[1017]	valid_0's l1: 0.256254	valid_0's l2: 0.13216
[1018]	valid_0's l1: 0.256267	valid_0's l2: 0.132173
[1019]	valid_0's l1: 0.256252	valid_0's l2: 0.132158
[1020]	valid_0's l1: 0.256235	valid_0's l2: 0.132143
[1021]	valid_0's l1: 0.256227	valid_0's l2: 0.132131
[1022]	valid_0's l1: 0.256228	valid_0's l2: 0.132131
[1023]	valid_0's l1: 0.256236	valid_0's l2: 0.13214
[1024]	valid_0's l1: 0.256224	valid

### Lasso CV

In [45]:
from sklearn.linear_model import  LassoCV
model5 = LassoCV().fit(X_train, Y_train)
prediction5 = model5.predict(X_test)
error(Y_test, prediction5)

Root Mean Square Error (RMSE)= 0.257 
Mean Absolute Error (MAE)= 0.354 
R^2 = 0.586


## Ridge

In [46]:
from sklearn.linear_model import Ridge
model6 = Ridge(alpha=1).fit(X_train, Y_train)
prediction6 = model6.predict(X_test)
error(Y_test, prediction6)

Root Mean Square Error (RMSE)= 0.257 
Mean Absolute Error (MAE)= 0.354 
R^2 = 0.587


## Linear

In [47]:
from sklearn.linear_model import LinearRegression
model7 = LinearRegression().fit(X_train, Y_train)
prediction7 = model7.predict(X_test)
error(Y_test, prediction7)

Root Mean Square Error (RMSE)= 0.257 
Mean Absolute Error (MAE)= 0.354 
R^2 = 0.587


# Elastic-Net

In [48]:
from sklearn.linear_model import ElasticNet
model8 = ElasticNet(alpha=1, l1_ratio=0.5).fit(X_train, Y_train)
prediction8 = model8.predict(X_test)
error(Y_test, prediction8)

Root Mean Square Error (RMSE)= 0.616 
Mean Absolute Error (MAE)= 0.625 
R^2 = 0.009


## XGboost

In [49]:
import xgboost as xgb
model9 = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.9,
                          learning_rate = 0.9, max_depth = 10, alpha = 30, n_estimators = 30).fit(X_train,Y_train)
prediction9 = model9.predict(X_test)
error(Y_test, prediction9)

Root Mean Square Error (RMSE)= 0.155 
Mean Absolute Error (MAE)= 0.266 
R^2 = 0.750


# Tuning of Hyperparameters

## Tuning of Ligthlgb

In [51]:
!pip install optuna

     |████████████████████████████████| 153kB 8.6MB/s 
     |████████████████████████████████| 1.1MB 17.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 112kB 38.5MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=2cabd66e85b27993912a623f55ff8e3de5234d683f14352af4745518bd25ca83
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.2.0-cp36-none-any.whl size=204611 sha256=42c2b3ffdb38fe2a8f86e8c6edf10536da52bc0483dc79983f4c244186abcf98
  Stored in directory: /root/.cache/pip/wheels/74/

In [52]:
import numpy as np
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import optuna.integration.lightgbm as lgb


dtrain = lgb.Dataset(X_train, label=Y_train)
dval = lgb.Dataset(X_test, label=Y_test)

params = {
    "objective": "regression",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    'metric':  'l2'
}

best_params, tuning_history = dict(), list()

model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    best_params=best_params,
    tuning_history=tuning_history,
    verbose_eval=600,
    early_stopping_rounds=100,
)

prediction = model.predict(X_test, num_iteration=model.best_iteration)



tune_feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds.
[600]	training's l2: 0.110777	valid_1's l2: 0.136411


tune_feature_fraction, val_score: 0.132871:  14%|#4        | 1/7 [00:03<00:21,  3.58s/it]

Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.100582	valid_1's l2: 0.132871


[I 2020-03-22 04:42:11,099] Finished trial#0 resulted in value: 0.13287119113087528. Current best value is 0.13287119113087528 with parameters: {'feature_fraction': 0.4}.
tune_feature_fraction, val_score: 0.132871:  14%|#4        | 1/7 [00:03<00:21,  3.58s/it]

...

tune_min_child_samples, val_score: 0.119332:  60%|######    | 3/5 [00:18<00:12,  6.14s/it]

Training until validation scores don't improve for 100 rounds.
[600]	training's l2: 0.064117	valid_1's l2: 0.12547
Early stopping, best iteration is:
[754]	training's l2: 0.058723	valid_1's l2: 0.124933


tune_min_child_samples, val_score: 0.119332:  80%|########  | 4/5 [00:26<00:06,  6.62s/it]

Training until validation scores don't improve for 100 rounds.
[600]	training's l2: 0.0729386	valid_1's l2: 0.126443
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.0609215	valid_1's l2: 0.12554


tune_min_child_samples, val_score: 0.119332: 100%|##########| 5/5 [00:37<00:00,  7.58s/it]


In [53]:
print("Number of finished trials: {}".format(len(tuning_history)))
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))


error(Y_test, prediction)

Number of finished trials: 65
Best params: {'lambda_l1': 0.45476949522521215, 'lambda_l2': 0.00045451482766904625, 'num_leaves': 148, 'feature_fraction': 0.92, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 5}
  Params: 
    lambda_l1: 0.45476949522521215
    lambda_l2: 0.00045451482766904625
    num_leaves: 148
    feature_fraction: 0.92
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 5
Root Mean Square Error (RMSE)= 0.119 
Mean Absolute Error (MAE)= 0.233 
R^2 = 0.808


## Tunning of XGBRegresor

In [54]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor

params = {
        'min_child_weight': [1, 5, 6, 7],
        'gamma': [5, 6, 7, 8, 9],
        'alpha':[50, 100, 150],
        'subsample': [ 0.7, 0.9, 1.0],
        'colsample_bytree': [0.75 ,0.8, 0.9],
        'max_depth': [29, 30, 32],
        'random_state' : np.random.randint(10000, size=20)
        }
xgb = XGBRegressor(learning_rate=0.9, n_estimators=30, objective='reg:squarederror')

skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 42)

random_search_xb = RandomizedSearchCV(xgb, param_distributions=params,
                                   n_iter= 5 , n_jobs=-1,cv = 2, verbose=3)

random_search_xb.fit(X_train, Y_train)


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   50.8s finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.9, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=30,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=0, reg_a...
                                        'colsample_bytree': [0.75, 0.8, 0.9],
                                        'gamma': [5, 6, 7, 8, 9],
                                        'max

In [55]:
best_params_xboosts = random_search_xb.best_params_
print('\n Best hyperparameters:',best_params_xboosts)


 Best hyperparameters: {'subsample': 1.0, 'random_state': 3635, 'min_child_weight': 5, 'max_depth': 29, 'gamma': 8, 'colsample_bytree': 0.9, 'alpha': 150}


In [56]:
ypred = random_search_xb.predict(X_test)
error(Y_test, ypred)

Root Mean Square Error (RMSE)= 0.158 
Mean Absolute Error (MAE)= 0.280 
R^2 = 0.746


## Tuning ExtraTrees

In [57]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
params = {
        'max_depth': [29, 30, 32],
        'random_state' : np.random.randint(10000, size=20),
        'max_features' : ['auto', 'sqrt', 'log2'],
        }
extr = ExtraTreeRegressor(splitter='best')

skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 42)

random_search_et = RandomizedSearchCV(extr, param_distributions=params,
                                   n_iter= 5 , n_jobs=-1,cv = 2, verbose=3)

random_search_et.fit(X_train, Y_train)


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.0s finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=ExtraTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                                max_depth=None,
                                                max_features='auto',
                                                max_leaf_nodes=None,
                                                min_impurity_decrease=0.0,
                                                min_impurity_split=None,
                                                min_samples_leaf=1,
                                                min_samples_split=2,
                                                min_weight_fraction_leaf=0.0,
                                                random_state=None,
                                                splitter='best'),
                   iid='deprecated', n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [29, 30, 32],
                                        'max_features':

In [58]:
best_params_et = random_search_et.best_params_
print('\n Best hyperparameters:',best_params_et)


 Best hyperparameters: {'random_state': 9502, 'max_features': 'log2', 'max_depth': 30}


In [59]:
ypred = random_search_et.predict(X_test)
error(Y_test, ypred)

Root Mean Square Error (RMSE)= 0.232 
Mean Absolute Error (MAE)= 0.297 
R^2 = 0.627


### Tuning BaggingRegresor

In [66]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
params = {
        'random_state' : np.random.randint(10000, size=20),
        'n_estimators' : np.random.randint(100, size=20),
        }
reg = BaggingRegressor()

skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 42)

random_search_b = RandomizedSearchCV(reg, param_distributions=params,
                                   n_iter= 1 ,cv = 2,  n_jobs=-1, verbose=3)

random_search_b.fit(X_train, Y_train)


Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    5.6s finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=BaggingRegressor(base_estimator=None,
                                              bootstrap=True,
                                              bootstrap_features=False,
                                              max_features=1.0, max_samples=1.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
                   iid='deprecated', n_iter=1, n_jobs=-1,
                   param_distributions={'n_estimators': array([24, 26, 74, 34, 46, 55, 64, 73, 60, 33, 32, 67, 72, 68, 13, 82, 72,
       27, 74, 66]),
                                        'random_state': array([ 997, 7075,   33,  709, 7537,  202, 3364, 9735, 5046, 3140, 3980,
       8397,  736, 6471, 7149, 4996, 5474, 5148, 5035, 99

In [67]:
best_params_b = random_search_b.best_params_
print('\n Best hyperparameters:',best_params_b)


 Best hyperparameters: {'random_state': 5035, 'n_estimators': 27}


In [68]:
ypred = random_search_b.predict(X_test)
error(Y_test, ypred)

Root Mean Square Error (RMSE)= 0.127 
Mean Absolute Error (MAE)= 0.234 
R^2 = 0.795


# Mixing of Best Models 

###  mixing xboost and bagging

In [60]:
bes_xgb = XGBRegressor(subsample = 0.9, random_state = 3061,
                       min_child_weight = 7, max_depth =  32,
                       gamma =  7,colsample_bytree = 0.8,
                       alpha = 150, learning_rate=0.9, 
                       n_estimators=30)#, objective='reg:squarederror')
baaggin = BaggingRegressor(base_estimator=bes_xgb, random_state = 5627, n_estimators = 10)
baaggin.fit(X_train, Y_train)

[04:50:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:51:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:51:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:51:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:51:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:51:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

BaggingRegressor(base_estimator=XGBRegressor(alpha=150, base_score=0.5,
                                             booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=0.8, gamma=7,
                                             importance_type='gain',
                                             learning_rate=0.9,
                                             max_delta_step=0, max_depth=32,
                                             min_child_weight=7, missing=None,
                                             n_estimators=30, n_jobs=1,
                                             nthread=None,
                                             objective='reg:linear',
                                             random_state=3061, reg_alpha=0,
                                             reg_lambda=1, scale_pos_weight=1,
             

In [61]:
ypred = baaggin.predict(X_test)
error(Y_test, ypred)

Root Mean Square Error (RMSE)= 0.138 
Mean Absolute Error (MAE)= 0.263 
R^2 = 0.778


### mixing bagging and extra_tree 

In [64]:
best_extr = ExtraTreeRegressor(random_state = 9502, max_features = 'log2', max_depth = 30, splitter='best')
baaggin2 = BaggingRegressor(base_estimator=best_extr, random_state = 5627, n_estimators = 10)
baaggin2.fit(X_train, Y_train)

BaggingRegressor(base_estimator=ExtraTreeRegressor(ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=30,
                                                   max_features='log2',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   random_state=9502,
                                                   splitter='best'),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=1.0, n_estimators=10, n_jobs=None,

In [65]:
ypred = baaggin2.predict(X_test)
error(Y_test, ypred)

Root Mean Square Error (RMSE)= 0.133 
Mean Absolute Error (MAE)= 0.243 
R^2 = 0.786
